Code for big trajectory optimization -- working off of PSET with planning around asteroids

In [ ]:
import numpy as np
from functools import partial

from pydrake.all import (
    AddDefaultVisualization,
    DiscreteContactApproximation,
    PidController,
    RobotDiagramBuilder,
    Simulator,
    StartMeshcat,
    AddMultibodyPlantSceneGraph,
    AddUnitQuaternionConstraintOnPlant,
    AutoDiffXd,
    DiagramBuilder,
    ExtractGradient,
    ExtractValue,
    InitializeAutoDiff,
    JacobianWrtVariable,
    JointIndex,
    MathematicalProgram,
    MeshcatVisualizer,
    OrientationConstraint,
    Parser,
    PiecewisePolynomial,
    PositionConstraint,
    RotationMatrix,
    SnoptSolver,
    Solve,
    eq,
    namedview,
)

import matplotlib.pyplot as plt
from underactuated import ConfigureParser, running_as_notebook
from underactuated.multibody import MakePidStateProjectionMatrix    


In [ ]:
#meshcat + create prog
meshcat = StartMeshcat()

In [ ]:
prog = MathematicalProgram() 
N = 150

In [ ]:
#define dynamics
com = prog.NewContinuousVariables(3, N, "com")
comdot = prog.NewContinuousVariables(3, N, "comdot")
comddot = prog.NewContinuousVariables(3, N - 1, "comddot")

In [ ]:
class Spot:
    """stores state of spot"""
    def __init__(self):
        self.com = None
        self.comdot = None
        self.comdotdot = None
        self.context = None
    
    def set_states(self, state_dict):
        self.com = state_dict['com']
        self.comdot = state_dict['comdot']
        self.comddot = state_dict['comddot']
        self.context = state_dict['context']


In [ ]:
class Obstacle:
    """obstacle w/ location and radius"""
    def __init__(self, x, y, rad):
        self.pos = (x, y)
        self.rad = rad

In [ ]:
traj_prog = MathematicalProgram()
time_steps = 150

position = traj_prog.NewContinuousVariables(time_steps + 1, 2, "position")
velocity = traj_prog.NewContinuousVariables(time_steps + 1, 2, "velocity")
acceleration = traj_prog.NewContinuousVariables(time_steps + 1, 2, "acceleration")

class Environment:
    """environment w/ multiple obstacles"""
    def __init__(self, goal, N=50):
        self.obstacles = []
        self.spot_rad = 0.8 #implement radius from spot CoM to be away from obstacle
        self.N = N #num_timesteps
        self.goal = goal
        
    def add_obstacle(self, obs):
        """adds obstacle"""
        if self.distance_from(obs.pos) > obs.rad:
            if obs.pos[0] >= 5 and obs.pos[0] <= 60:
                self.obstacles.append(obs)
    
    def _residuals(self, obs, n):
        return position[n, :] - obs.pos

    def generate_constraints(self):
        """generates necessary constraints for obstacles"""
        for obs in self.obstacles:
            for n in range(self.N):
                residual = self._residuals(obs, n)
                traj_prog.AddConstraint(residual[0]**2 + residual[1]**2 >= (obs.rad + self.spot_rad)**2)
    
    def _euclidean_dist(self, pos1, pos2):
        x1, y1 = pos1 
        x2, y2 = pos2
        return ((x1 - x2)**2 + (y1 - y2)**2)**0.5

    def distance_from(self, point):
        """calculates distance of given point from nearest obstacle"""
        if not self.obstacles:
            return np.inf
        
        distances = [self._euclidean_dist(obs.pos, point) for obs in self.obstacles]
        return min(distances)  

    

In [ ]:
#visualization helpers

def plot_circle(center, radius, *args, **kwargs):
    """taken from orbital transfer notebook"""
    # discretize angle
    angle = np.linspace(0, 2 * np.pi)

    # plot circle
    plt.plot(
        center[0] + radius * np.cos(angle),
        center[1] + radius * np.sin(angle),
        *args,
        **kwargs
    )

def plot_env(env):
    for obs in env.obstacles:
        plt.scatter(*obs.pos, s=10, c='red')
        
        plot_circle(obs.pos, obs.rad, color = 'red', linestyle='--', linewidth = 0.5)
        plot_circle(obs.pos, obs.rad + env.spot_rad, color='blue', linestyle='--', linewidth = 0.5)
    plt.scatter(*env.goal, s=50, c='purple')
    plt.scatter(0, 0, s=50, c='purple')
    plt.grid(True)
    plt.gca().set_aspect("equal")
    ax = plt.gca()
    ax.set_xlim([0, 100])
    ax.set_ylim([-50, 50])

def plot_trajectory(trajectory):
    plt.plot(trajectory.T[0], trajectory.T[1], color="k", label="Rocket trajectory")
    plt.scatter(trajectory[0, 0], trajectory[0, 1], color="k")


In [ ]:
#add obstacles
obs1 = Obstacle(30, 3, 6)
obs2 = Obstacle(20, -10, 2)
obs3 = Obstacle(40, -25, 2)
obs4 = Obstacle(40, -8, 4)
obs5 = Obstacle(60, 0, 5)
env = Environment((80, 0), N=time_steps)
env.add_obstacle(obs1)
env.add_obstacle(obs2)
env.add_obstacle(obs3)
env.add_obstacle(obs4)
env.add_obstacle(obs5)
plot_env(env)
plt.figure()

In [ ]:
#build initial guess
def built_init_guess(pos_init, pos_final, time_steps, h):
    np.random.seed(0)

    time_limits = [0.0, time_steps * h]
    position_limits = np.column_stack((pos_init, pos_final))
    state_limits = np.vstack((position_limits, np.zeros((2, 2))))

    # linear interpolation in state
    state = PiecewisePolynomial.FirstOrderHold(time_limits, state_limits)

    # sample state on the time grid and add small random noise
    state_guess = np.vstack(
        [state.value(t * h).T for t in range(time_steps + 1)]
    )
    state_guess += np.random.rand(*state_guess.shape) * 5e-6

    return state_guess

In [ ]:
#optimization to find trajectory (minimize need to change direction --> goal is to have primarily straight sections)
h = 0.05
end_goal = env.goal

#starting and ending position constraint

#euler dynamics constraint
for ts in range(time_steps):
    traj_prog.AddConstraint(eq(position[ts + 1, :], position[ts, :] + h * velocity[ts, :]))
    traj_prog.AddConstraint(eq(velocity[ts + 1, :], velocity[ts, :] + h * acceleration[ts, :]))

for n in [0, 1]:
    traj_prog.AddLinearConstraint(position[0, n] == 0) #start at origin
    traj_prog.AddLinearConstraint(velocity[0, n] == 0)


traj_prog.AddLinearConstraint(position[-1, 0] == end_goal[0]) #end at (80, 0)
traj_prog.AddLinearConstraint(position[-1, 1] == end_goal[1])

for n in [0, 1]:
    traj_prog.AddLinearConstraint(velocity[-1, n] == 0)
    traj_prog.AddLinearConstraint(acceleration[-1, n] == 0)

env.generate_constraints()


traj_prog.AddBoundingBoxConstraint(0, 80, position[n, 0])
traj_prog.AddBoundingBoxConstraint(-50, 50, position[n, 1])


#bounding box on velocity

traj_prog.AddBoundingBoxConstraint(-1, 1, acceleration[ts, n])

#set initial guess
start = (0, 0)
guess = built_init_guess(start, end_goal, time_steps, h)
traj_prog.SetInitialGuess(position, guess[:, :2])
traj_prog.SetInitialGuess(velocity, guess[:, 2:])

traj_prog.AddCost(h*sum(t.dot(t) for t in acceleration))

step_dist = []
for n in range(1, N):
    x_dist = (position[n, 0]-position[n-1, 0])**2
    y_dist = (position[n, 1]-position[n-1, 1])**2
    step_dist.append(x_dist + y_dist)
traj_prog.AddCost(sum(step_dist))
traj_prog.AddCost(8*time_steps/5*sum(pos**2 for pos in position[:time_steps//3, 1]))

# start_dist = []
# for n in range(1, N):
#     x_dist = (position[n, 0]-start[0])**2
#     y_dist = (position[n, 1]-start[1])**2
#     start_dist.append(x_dist + y_dist)
# traj_prog.AddCost(sum(start_dist))

# goal_dist = []
# for n in range(0, N):
#     x_dist = (position[n, 0]-end_goal[0])**2
#     y_dist = (position[n, 1]-end_goal[1])**2
#     goal_dist.append(x_dist + y_dist)
# traj_prog.AddCost(sum(goal_dist))

In [ ]:
snopt = SnoptSolver()

result = snopt.Solve(traj_prog)
print(result.is_success())
positions = result.GetSolution(position)
velocities = result.GetSolution(velocity)

plot_trajectory(positions)
plot_env(env)
plt.figure()

In [ ]:
#find trajectory waypoints, may need waypoint class 
#instantiate trajectory
#generate trajectory
foward_break = None
for pos in positions:
    if abs(pos[1]) > 1:
        forward_break = pos[0]
        break
print(forward_break)
far_len = 0
far_obs = None
for obs in env.obstacles:
    if obs.pos[0] > far_len:
        far_obs = obs
        far_len = obs.pos[0]

split_point = far_len + far_obs.rad
print(split_point)

In [ ]:
#animate trajectory

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=545d651b-e7e0-4cf4-bfbe-97d6a3f869f8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>